tasks:
- take the data with cleaned missingness and encode columns.
- to encode columns, join original column names using `master` to `data_miss`.
- then used the original column names to retrieve the decodings from `lookup`


In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plot

In [2]:
# load data
path = r"../../data/interim/Lookup_V2variables_share_utf8.csv"
lookup1 = pd.read_csv(path, sep=";")

path = r"../../data/interim/Lookup20200804212715_utf8.csv"
lookup2 = pd.read_csv(path, sep=";", low_memory=False) # low memory prevents dtype problems

path = r"../../data/interim/Mastertabelle_utf8.csv"
master = pd.read_csv(path, sep=";")

print(f"lookup1.shape {lookup1.shape}")
print(f"lookup2.shape {lookup2.shape}")
print(f"master.shape {master.shape}")

lookup = pd.concat([lookup1,lookup2], axis=1)
del lookup1, lookup2
print(f"lookup.shape {lookup.shape}")

data_miss = pd.read_pickle("../../data/interim/data_cleaned_missingness.pkl")
print(f"data_miss.shape {data_miss.shape}")


lookup1.shape (25, 142)
lookup2.shape (1053, 4854)
master.shape (2588, 9)
lookup.shape (1053, 4996)
data_miss.shape (6241, 222)


# exceptions



In [3]:
# TODO FIXME

In [4]:
# BL_MonthFirstDiagnosis Spalte fehlerhaft
data_miss = data_miss.drop(columns = "BL_MonthFirstDiagnosis")

In [5]:
lookup = lookup.drop(columns = ["dupl1_v_2061","dupl1_v_2061_B"])

# clean lookup table

In [6]:
print(f"number of columns in lookup: {len(lookup.columns)}")
print()
print(f"number of unique columns in lookup: {len(lookup.columns)}")
print()
non_unique_cols = lookup.columns.value_counts()[lookup.columns.value_counts()>1]
print(f"non-unique columns:\n{non_unique_cols}")

number of columns in lookup: 4994

number of unique columns in lookup: 4994

non-unique columns:
v_345_B     2
v_1992_B    2
v_1992      2
v_345       2
dtype: int64


In [7]:
# preserve the column which has more values
def lookup_remove_duplicates(lookup):
    print(f"original shape = {lookup.shape}")
    cols = lookup.columns
    duplicates = cols.value_counts()[cols.value_counts()>1]
    print(f"duplicates = \n{duplicates}")
    col_numbers = [_ for _ in range(lookup.shape[1])]    
    # iterate through the non-unique columns
    for col in list(duplicates.index):
        idx = list(np.where(cols == col)[0])
        idx_counts = []        
        # counter number of unique values per non-unique column
        for i in idx:
            idx_counts.append(lookup.iloc[:,i].value_counts().count())
        # cherry pick column with most unique values
        idx_best = np.where(idx_counts == np.max(idx_counts))
        # if many best exist, take the smallest index 
        idx_best = np.min(idx_best)
        # all but best indices
        idx_notbest = idx.pop(idx_best)
        # remove all but best indices from column indices
        col_numbers.remove(idx_notbest)
    return lookup.iloc[:,col_numbers].copy()

In [8]:
lookup_cleaned = lookup_remove_duplicates(lookup)
lookup_cleaned.to_pickle("../../data/interim/lookup_cleaned.pkl", compression = "zip")
del lookup

original shape = (1053, 4994)
duplicates = 
v_345_B     2
v_1992_B    2
v_1992      2
v_345       2
dtype: int64


# merge old names to new names

In [9]:
# merge old names to new names
colmap = pd.DataFrame({"new": data_miss.columns})
master.rename(columns={
    'Neue.Variablennamen_V2' : "new",
    'OriginalVariablenName' : "old"
    }, inplace=True)

colmap_merged = colmap.merge(
    master[['old','new']], how="left", left_on="new", right_on="new")
print(f"colmap.shape = {colmap.shape}")
print(f"colmap_merged.shape = {colmap_merged.shape}")
print(colmap_merged.head())

colmap.shape = (221, 1)
colmap_merged.shape = (228, 2)
                          new                         old
0                      BL_Age                      v_3176
1                   BL_Gender                       v_341
2                BL_Ethnicity                       v_342
3  BL_AdmissionComputedNewCat  BL_AdmissionComputedNewCat
4       BL_EmergencyAdmission                      v_3078


In [10]:
# shapes are not consistent, check for duplicates
not_unique = colmap_merged.old.value_counts() == 2
not_unique = not_unique.index[not_unique]
print(not_unique)

Index(['v_1732', 'v_1731', 'v_1733', 'v_1727', 'v_1734', 'v_1730', 'v_1728'], dtype='object')


In [11]:
# inspect duplicates
colmap_merged[colmap_merged.old.isin(not_unique)]

,new,old
140,BL_Vital_SystolicBP,v_1727
141,BL_Vital_SystolicBP,v_1727
142,BL_Vital_DiastolicBP,v_1728
143,BL_Vital_DiastolicBP,v_1728
144,BL_Vital_paCO2,v_1732
145,BL_Vital_paCO2,v_1732
146,BL_Vital_paO2,v_1731
147,BL_Vital_paO2,v_1731
149,BL_Vital_RespRate,v_1734
150,BL_Vital_RespRate,v_1734


In [12]:
# all non-unique representations are only duplicates. remove duplicates
colmap_merged = colmap_merged.drop_duplicates()
print(f"colmap_merged.shape = {colmap_merged.shape}")

colmap_merged.shape = (221, 2)


In [13]:
def check_merged_names(colmap_merged, lookup):
    """check if all merged names are in to lookup table"""
    in_lookup = []
    for col in colmap_merged.old:
        if col in lookup.columns:
            in_lookup.append(True)
        else:
            in_lookup.append(False)

    print("\n")
    print(f"from {len(in_lookup)} columns in colmap_merged, {sum(in_lookup)} are in lookup")
    print("\n")
    print(f"columns from colmap_merged which are not in lookup: \n{colmap_merged[~np.array(in_lookup)]}")

check_merged_names(colmap_merged, lookup_cleaned)



from 221 columns in colmap_merged, 205 are in lookup


columns from colmap_merged which are not in lookup: 
                                 new                              old
3         BL_AdmissionComputedNewCat       BL_AdmissionComputedNewCat
5      BL_DaysInfectionContactNewCat    BL_DaysInfectionContactNewCat
8         BL_UCstartdayNewCategories       BL_UCstartdayNewCategories
10        BL_COstartdayNewCategories       BL_COstartdayNewCategories
12        BL_CRstartdayNewCategories       BL_CRstartdayNewCategories
14        BL_REstartdayNewCategories       BL_REstartdayNewCategories
15   BL_Duration_InpatientstayNewCat  BL_Duration_InpatientstayNewCat
16         BL_Duration_ICUStayNewCat        BL_Duration_ICUStayNewCat
17              BL_Duration_IMCUstay                           v_3766
18     BL_Duration_VentilationNewCat    BL_Duration_VentilationNewCat
19                           BL_ECMO                           v_3767
20      BL_ObservationalPeriodNewCat     BL_Observ

In [14]:
#                                  new                              old
# 3         BL_AdmissionComputedNewCat       BL_AdmissionComputedNewCat -> time
# 6      BL_DaysInfectionContactNewCat    BL_DaysInfectionContactNewCat -> time
# 9         BL_UCstartdayNewCategories       BL_UCstartdayNewCategories -> time
# 11        BL_COstartdayNewCategories       BL_COstartdayNewCategories -> time
# 13        BL_CRstartdayNewCategories       BL_CRstartdayNewCategories -> time
# 15        BL_REstartdayNewCategories       BL_REstartdayNewCategories -> time
# 16   BL_Duration_InpatientstayNewCat  BL_Duration_InpatientstayNewCat -> time
# 17         BL_Duration_ICUStayNewCat        BL_Duration_ICUStayNewCat -> time
# 18              BL_Duration_IMCUstay                           v_3766 -> time
# 19     BL_Duration_VentilationNewCat    BL_Duration_VentilationNewCat -> time
# 20                           BL_ECMO                           v_3767 -> boolean
# 21      BL_ObservationalPeriodNewCat     BL_ObservationalPeriodNewCat -> time
# 149                   BL_Vital_Pulse                   BL_Vital_Pulse -> replace old with "BL_Vital_PulseV2" # DONE
# 157              BL_Function_HeartEF              BL_Function_HeartEF -> replace old with "BL_Function_HeartEFV2" # DONE
# 179                  BL_igG_positive                     dupl1_v_3018 -> unknown # TODO FIXME

## fix wrong assignments

In [15]:
# 149                   BL_Vital_Pulse                   BL_Vital_Pulse -> replace old with "BL_Vital_PulseV2"
# 157              BL_Function_HeartEF              BL_Function_HeartEF -> replace old with "BL_Function_HeartEFV2"
colmap_merged.old[colmap_merged.old == "BL_Vital_Pulse"] = "BL_Vital_PulseV2"
colmap_merged.old[colmap_merged.old == "BL_Function_HeartEF"] = "BL_Function_HeartEFV2"

In [16]:
check_merged_names(colmap_merged, lookup_cleaned)



from 221 columns in colmap_merged, 207 are in lookup


columns from colmap_merged which are not in lookup: 
                                 new                              old
3         BL_AdmissionComputedNewCat       BL_AdmissionComputedNewCat
5      BL_DaysInfectionContactNewCat    BL_DaysInfectionContactNewCat
8         BL_UCstartdayNewCategories       BL_UCstartdayNewCategories
10        BL_COstartdayNewCategories       BL_COstartdayNewCategories
12        BL_CRstartdayNewCategories       BL_CRstartdayNewCategories
14        BL_REstartdayNewCategories       BL_REstartdayNewCategories
15   BL_Duration_InpatientstayNewCat  BL_Duration_InpatientstayNewCat
16         BL_Duration_ICUStayNewCat        BL_Duration_ICUStayNewCat
17              BL_Duration_IMCUstay                           v_3766
18     BL_Duration_VentilationNewCat    BL_Duration_VentilationNewCat
19                           BL_ECMO                           v_3767
20      BL_ObservationalPeriodNewCat     BL_Observ

## clean **time** related columns

take the **most conservative** approach


-  3         BL_AdmissionComputedNewCat       BL_AdmissionComputedNewCat -> time
-  6      BL_DaysInfectionContactNewCat    BL_DaysInfectionContactNewCat -> time
-  9         BL_UCstartdayNewCategories       BL_UCstartdayNewCategories -> time
-  11        BL_COstartdayNewCategories       BL_COstartdayNewCategories -> time
-  13        BL_CRstartdayNewCategories       BL_CRstartdayNewCategories -> time
-  15        BL_REstartdayNewCategories       BL_REstartdayNewCategories -> time
-  16   BL_Duration_InpatientstayNewCat  BL_Duration_InpatientstayNewCat -> time
-  17         BL_Duration_ICUStayNewCat        BL_Duration_ICUStayNewCat -> time
-  18              BL_Duration_IMCUstay                           v_3766 -> time
-  19     BL_Duration_VentilationNewCat    BL_Duration_VentilationNewCat -> time
-  21      BL_ObservationalPeriodNewCat     BL_ObservationalPeriodNewCat -> time


### 3         BL_AdmissionComputedNewCat       BL_AdmissionComputedNewCat -> time

In [17]:
data_miss["BL_AdmissionComputedNewCat"].value_counts()

0            2599
-1            467
1             284
2             226
7             194
5             184
3             183
4             173
6             138
-2            113
8             100
9              90
10             87
-7             79
[-21;-16]      70
-3             67
-5             65
11             62
-6             55
-4             55
[18;21]        53
[22;28]        47
-9             47
-8             43
-10            39
14             37
-14            37
12             34
13             32
-12            29
[-28;-22]      26
15             25
-11            22
<=-36          21
[-35;-29]      20
-15            19
-13            16
16             16
[29;35]        13
17             12
>=50           10
Name: BL_AdmissionComputedNewCat, dtype: int64

In [18]:
BL_AdmissionComputedNewCat_replace = {
    "[-21;-16]" : -21,
    "[18;21]" : 18,
    "[22;28]" : 22,
    "[-28;-22]" : -28,
    "<=-36" : -36,
    "[-35;-29]" : -35,
    "[29;35]" : 29,
    ">=50" : 50
    }


In [19]:
data_miss["BL_AdmissionComputedNewCat"].replace(BL_AdmissionComputedNewCat_replace).value_counts()

0      2599
-1      467
1       284
2       226
7       194
5       184
3       183
4       173
6       138
-2      113
8       100
9        90
10       87
-7       79
-21      70
-3       67
-5       65
11       62
-6       55
-4       55
18       53
22       47
-9       47
-8       43
-10      39
14       37
-14      37
12       34
13       32
-12      29
-28      26
15       25
-11      22
-36      21
-35      20
-15      19
-13      16
16       16
29       13
17       12
50       10
Name: BL_AdmissionComputedNewCat, dtype: int64

In [20]:
data_miss["BL_AdmissionComputedNewCat"].replace(BL_AdmissionComputedNewCat_replace, inplace=True)

### 6      BL_DaysInfectionContactNewCat    BL_DaysInfectionContactNewCat -> time

In [21]:
col = "BL_DaysInfectionContactNewCat"
data_miss[col].value_counts()

7          119
5           90
14          60
6           52
10          49
[15;21]     48
8           45
4           42
3           39
9           39
1           35
2           35
12          25
0           23
>=22        19
11          18
13          15
Name: BL_DaysInfectionContactNewCat, dtype: int64

In [22]:
to_replace = {
    "[15;21]" : 15,
    ">=22": 22
    }

In [23]:
data_miss[col].replace(to_replace).value_counts()

7     119
5      90
14     60
6      52
10     49
15     48
8      45
4      42
3      39
9      39
2      35
1      35
12     25
0      23
22     19
11     18
13     15
Name: BL_DaysInfectionContactNewCat, dtype: int64

In [24]:
data_miss[col].replace(to_replace, inplace=True)

### 9         BL_UCstartdayNewCategories       BL_UCstartdayNewCategories -> time

In [25]:
col = "BL_UCstartdayNewCategories"
data_miss[col].value_counts()

0            2137
-1            423
-2            388
-3            363
-7            355
-5            260
-4            258
-6            167
-10           134
-14           123
-8            110
-9             78
[-21;-16]      51
-11            50
1              41
-12            35
[8;14]         31
-13            31
<=-22          30
3              23
[6;7]          19
2              15
>=15           14
5              12
-15            12
4               7
Name: BL_UCstartdayNewCategories, dtype: int64

In [26]:
to_replace = {
"[-21;-16]" : -21,
"[8;14]" : 8,
"<=-22" : -22,
"[6;7]" : 6,
">=15" : 15
}


In [27]:
data_miss[col].replace(to_replace).value_counts()

0      2137
-1      423
-2      388
-3      363
-7      355
-5      260
-4      258
-6      167
-10     134
-14     123
-8      110
-9       78
-21      51
-11      50
1        41
-12      35
8        31
-13      31
-22      30
3        23
6        19
2        15
15       14
5        12
-15      12
4         7
Name: BL_UCstartdayNewCategories, dtype: int64

In [28]:
data_miss[col].replace(to_replace, inplace=True)

### 11        BL_COstartdayNewCategories       BL_COstartdayNewCategories -> time

In [29]:
col = "BL_COstartdayNewCategories"
data_miss[col].value_counts()


0          1147
1           234
2           185
3           161
-1          131
4           126
5           124
7           108
6            90
8            83
9            63
10           47
[15;21]      42
11           42
-2           41
<=-8         35
[-7;-6]      25
14           24
12           24
-3           21
13           20
>=22         19
-5           14
-4           10
Name: BL_COstartdayNewCategories, dtype: int64

In [30]:
to_replace = {
    "[15;21]" : 15,
    ">=22" : 22,
    "<=-8" : -8,
    "[-7;-6]" : -7
    }

In [31]:
data_miss[col].replace(to_replace).value_counts()

0     1147
1      234
2      185
3      161
-1     131
4      126
5      124
7      108
6       90
8       83
9       63
10      47
11      42
15      42
-2      41
-8      35
-7      25
12      24
14      24
-3      21
13      20
22      19
-5      14
-4      10
Name: BL_COstartdayNewCategories, dtype: int64

In [32]:
data_miss[col].replace(to_replace, inplace=True)

### 13        BL_CRstartdayNewCategories       BL_CRstartdayNewCategories -> time

In [33]:
col = ["BL_CRstartdayNewCategories"]
data_miss[col].value_counts()


BL_CRstartdayNewCategories
0                             285
1                             134
2                             114
3                              85
5                              82
4                              78
6                              60
-1                             52
7                              48
[-7;-2]                        40
8                              37
[15;21]                        34
9                              29
10                             23
11                             19
<=-8                           18
12                             14
14                             13
13                             13
>=22                           12
dtype: int64

In [34]:
to_replace = {
    "[-7;-2]" : -7,
    "[15;21]" : 15,
    ">=22" : 22,
    "<=-8" : -8,
    }

In [35]:
data_miss[col].replace(to_replace).value_counts()

BL_CRstartdayNewCategories
0                             285
1                             134
2                             114
3                              85
5                              82
4                              78
6                              60
-1                             52
7                              48
-7                             40
8                              37
15                             34
9                              29
10                             23
11                             19
-8                             18
12                             14
13                             13
14                             13
22                             12
dtype: int64

In [36]:
data_miss[col].replace(to_replace, inplace=True)

C:\Users\tlinden\anaconda3\envs\coperimo_tests\lib\site-packages\pandas\core\frame.py:4383: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [37]:
data_miss[col] = data_miss[col].replace(to_replace)

In [38]:
data_miss[col].value_counts()

BL_CRstartdayNewCategories
0                             285
1                             134
2                             114
3                              85
5                              82
4                              78
6                              60
-1                             52
7                              48
-7                             40
8                              37
15                             34
9                              29
10                             23
11                             19
-8                             18
12                             14
13                             13
14                             13
22                             12
dtype: int64

### 15        BL_REstartdayNewCategories       BL_REstartdayNewCategories -> time

In [39]:
col = "BL_REstartdayNewCategories"
data_miss[col].value_counts()


1          233
14         226
6          166
8          163
5          161
7          159
12         154
9          136
10         135
11         133
4          128
15         120
3          118
13         117
2          103
16          76
17          73
18          71
20          59
[31;35]     58
21          54
[36;42]     54
19          53
23          39
22          37
24          34
26          28
28          27
25          27
[43;49]     27
<=0         26
>=57        24
30          21
27          19
[50;56]     16
29          14
Name: BL_REstartdayNewCategories, dtype: int64

In [40]:
to_replace = {
"[31;35]" : 31,
"[36;42]" : 36,
"[43;49]" : 43,
"<=0" : 0,
">=57" : 57,
"[50;56]" : 50
}



In [41]:
data_miss[col].replace(to_replace).value_counts()

1     233
14    226
6     166
8     163
5     161
7     159
12    154
9     136
10    135
11    133
4     128
15    120
3     118
13    117
2     103
16     76
17     73
18     71
20     59
31     58
36     54
21     54
19     53
23     39
22     37
24     34
26     28
25     27
28     27
43     27
0      26
57     24
30     21
27     19
50     16
29     14
Name: BL_REstartdayNewCategories, dtype: int64

In [42]:
data_miss[col].replace(to_replace, inplace=True)

### 16   BL_Duration_InpatientstayNewCat  BL_Duration_InpatientstayNewCat -> time

In [43]:
col = "BL_Duration_InpatientstayNewCat"
data_miss[col].value_counts()


0          499
6          352
7          347
4          330
3          327
5          325
2          309
8          288
10         272
9          272
11         223
1          215
12         198
14         191
13         164
15         154
16         137
17         114
21          97
20          96
19          94
18          88
[44;49]     83
22          78
23          71
26          62
[50;56]     60
25          59
24          52
28          52
30          50
27          48
[57;63]     44
36          39
31          38
>=78        37
40          34
35          34
29          34
33          28
37          27
[64;70]     26
34          24
38          22
32          20
39          19
43          17
41          16
[71;77]     15
42          14
Name: BL_Duration_InpatientstayNewCat, dtype: int64

In [44]:

to_replace = {
    "[44;49]" : 44,
    "[50;56]" : 50,
    "[57;63]" : 57,
    ">=78" : 78,
    "[64;70]" : 64,
    "[71;77]" : 71
    }

data_miss[col].replace(to_replace).value_counts()


0     499
6     352
7     347
4     330
3     327
5     325
2     309
8     288
10    272
9     272
11    223
1     215
12    198
14    191
13    164
15    154
16    137
17    114
21     97
20     96
19     94
18     88
44     83
22     78
23     71
26     62
50     60
25     59
28     52
24     52
30     50
27     48
57     44
36     39
31     38
78     37
35     34
29     34
40     34
33     28
37     27
64     26
34     24
38     22
32     20
39     19
43     17
41     16
71     15
42     14
Name: BL_Duration_InpatientstayNewCat, dtype: int64

In [45]:
data_miss[col].replace(to_replace, inplace=True)

### 17         BL_Duration_ICUStayNewCat        BL_Duration_ICUStayNewCat -> time

In [46]:
col = "BL_Duration_ICUStayNewCat"
data_miss[col].value_counts()

0          2806
3            98
1            97
2            94
4            76
6            72
5            66
7            61
9            60
[36;42]      53
8            52
11           45
[32;35]      44
12           44
17           44
10           41
18           40
13           39
14           38
[43;49]      34
15           33
19           31
[50;56]      30
22           27
16           25
24           24
23           24
>=64         23
27           21
20           19
28           17
21           17
25           16
[57;63]      15
30           13
29           13
26           11
31           11
Name: BL_Duration_ICUStayNewCat, dtype: int64

In [47]:
to_replace = {
    "[36;42]" : 36,
    "[32;35]" : 32,
    "[43;49]" : 43,
    "[50;56]" : 50,
    ">=64" : 64,
    "25" : 25,
    "[57;63]" : 57,
    }

data_miss[col].replace(to_replace).value_counts()

0     2806
3       98
1       97
2       94
4       76
6       72
5       66
7       61
9       60
36      53
8       52
11      45
32      44
17      44
12      44
10      41
18      40
13      39
14      38
43      34
15      33
19      31
50      30
22      27
16      25
24      24
23      24
64      23
27      21
20      19
28      17
21      17
25      16
57      15
30      13
29      13
26      11
31      11
Name: BL_Duration_ICUStayNewCat, dtype: int64

In [48]:
data_miss[col].replace(to_replace, inplace=True)

### 18              BL_Duration_IMCUstay                           v_3766 -> time

In [49]:
col = "BL_Duration_IMCUstay"
data_miss[col].value_counts()
# nothing to do

0     2322
>0     141
Name: BL_Duration_IMCUstay, dtype: int64

### 19     BL_Duration_VentilationNewCat    BL_Duration_VentilationNewCat -> time

In [50]:
col = "BL_Duration_VentilationNewCat"
data_miss[col].value_counts()


0          2977
1            66
[29;35]      65
5            61
7            58
9            54
6            54
3            53
[25;28]      50
8            47
4            46
11           42
[36;42]      37
12           36
14           35
10           34
[43;49]      34
2            31
15           30
13           29
17           26
18           25
20           24
>=57         21
22           19
19           18
24           18
23           17
16           17
[50;56]      15
21           14
Name: BL_Duration_VentilationNewCat, dtype: int64

In [51]:
to_replace = {
    "[29;35]" : 29,
    "[25;28]" : 25,
    "[36;42]" : 36,
    "[43;49]" : 43,
    ">=57" : 57,
    "[50;56]" : 50,
}

data_miss[col].replace(to_replace).value_counts()

0     2977
1       66
29      65
5       61
7       58
6       54
9       54
3       53
25      50
8       47
4       46
11      42
36      37
12      36
14      35
10      34
43      34
2       31
15      30
13      29
17      26
18      25
20      24
57      21
22      19
19      18
24      18
16      17
23      17
50      15
21      14
Name: BL_Duration_VentilationNewCat, dtype: int64

In [52]:
data_miss[col].replace(to_replace, inplace=True)

### 21      BL_ObservationalPeriodNewCat     BL_ObservationalPeriodNewCat -> time

In [53]:
col = "BL_ObservationalPeriodNewCat"
data_miss[col].value_counts()

1          389
7          325
5          306
6          293
8          292
14         291
10         288
4          272
2          266
3          260
11         246
9          244
12         237
15         209
13         208
16         186
17         141
21         130
18         123
19         122
22         105
20         105
[43;49]    103
24          89
25          81
23          77
30          71
26          71
27          66
[50;56]     64
28          56
>=78        55
29          49
[57;63]     44
36          40
31          39
40          36
33          35
35          35
32          28
37          28
34          26
[64;70]     26
[41;42]     25
38          23
39          21
[71;77]     13
Name: BL_ObservationalPeriodNewCat, dtype: int64

In [54]:
to_replace = {
    "[43;49]" : 43,
    "[50;56]" : 50,
    ">=78" : 78,
    "[57;63]" : 57,
    "[64;70]" : 64,
    "[41;42]" : 41,
    "[71;77]" : 71,
    }

data_miss[col].replace(to_replace).value_counts()

1     389
7     325
5     306
6     293
8     292
14    291
10    288
4     272
2     266
3     260
11    246
9     244
12    237
15    209
13    208
16    186
17    141
21    130
18    123
19    122
20    105
22    105
43    103
24     89
25     81
23     77
26     71
30     71
27     66
50     64
28     56
78     55
29     49
57     44
36     40
31     39
40     36
33     35
35     35
32     28
37     28
64     26
34     26
41     25
38     23
39     21
71     13
Name: BL_ObservationalPeriodNewCat, dtype: int64

In [55]:
data_miss[col].replace(to_replace, inplace=True)

### backcheck all cleaned columns

try to convert values to `int`. if this fails, go back and clean data

In [56]:
cols = [
    "BL_AdmissionComputedNewCat",
    "BL_DaysInfectionContactNewCat",
    "BL_UCstartdayNewCategories",
    "BL_COstartdayNewCategories",
    "BL_CRstartdayNewCategories",
    "BL_REstartdayNewCategories",
    "BL_Duration_InpatientstayNewCat",
    "BL_Duration_ICUStayNewCat",
    # "BL_Duration_IMCUstay", # categorical
    "BL_Duration_VentilationNewCat",
    "BL_ObservationalPeriodNewCat",
    ]

In [57]:
cols_to_clean = []
for col in cols:
    try:
        data_miss[col].value_counts().astype(int)
        print(f"col {col} -- PASSED")
    except ValueError:
        cols_to_clean.append(col)
        print(f"col {col} -- FAILED")

col BL_AdmissionComputedNewCat -- PASSED
col BL_DaysInfectionContactNewCat -- PASSED
col BL_UCstartdayNewCategories -- PASSED
col BL_COstartdayNewCategories -- PASSED
col BL_CRstartdayNewCategories -- PASSED
col BL_REstartdayNewCategories -- PASSED
col BL_Duration_InpatientstayNewCat -- PASSED
col BL_Duration_ICUStayNewCat -- PASSED
col BL_Duration_VentilationNewCat -- PASSED
col BL_ObservationalPeriodNewCat -- PASSED


# decode variables in `data_miss` with match in `lookup`

In [58]:
"""
pseudocode

create a copy of the data

for column in data:
    check if the column is in lookup and get the decoding
    replace the encodings with the decodings
        extract encoding column
        merge decoding
        overwrite encoding

"""

'\npseudocode\n\ncreate a copy of the data\n\nfor column in data:\n    check if the column is in lookup and get the decoding\n    replace the encodings with the decodings\n        extract encoding column\n        merge decoding\n        overwrite encoding\n\n'

In [59]:
data_miss.rename(dict(zip(colmap_merged.new,colmap_merged.old)), axis=1, inplace = True)

In [60]:
def decode_variables(x : pd.DataFrame, lookup) -> pd.DataFrame:
    x_ = x.copy()
    x_cols = x_.columns
    l_cols = lookup.columns
    for i, xc in enumerate(x_cols):
        if xc in l_cols:
            print(f"\n === {i} ===")
            # construct mapping df
            enc_col = xc
            dec_col = xc + "_B"
            print(f"enc_col = {enc_col}")
            print(f"dec_col = {dec_col}")
            enc_dec = pd.DataFrame({"enc" : lookup[enc_col], "dec" : lookup[dec_col]})
            enc_dec.dropna(inplace=True)
            # decode
            to_decode = pd.DataFrame({"enc" : x_[enc_col].copy()})

            print(to_decode)
            print()
            print(enc_dec)

            to_decode = to_decode.merge(enc_dec, how="left", on="enc")
            # replace encoding with decoding
            x_[enc_col] = to_decode["dec"]
    return x_


        

In [61]:
data_dec = decode_variables(data_miss, lookup_cleaned)

 ULN <= 20x ULN
5     6.0             > 20x ULN
6     7.0                 < LLN
7   999.0              Not done
8  1000.0               Unknown

 === 177 ===
enc_col = dupl1_v_885
dec_col = dupl1_v_885_B
         enc
0        1.0
1     1000.0
2     1000.0
3        1.0
4        1.0
...      ...
6452  1000.0
6453     1.0
6454   999.0
6455  1000.0
6456  1000.0

[6241 rows x 1 columns]

      enc                   dec
0     1.0   Normal  (LLN - ULN)
1     2.0       > ULN <= 2x ULN
2     3.0    > 2x ULN <= 5x ULN
3     4.0   > 5x ULN <= 10x ULN
4     5.0  > 10x ULN <= 20x ULN
5     6.0             > 20x ULN
6     7.0                 < LLN
7   999.0              Not done
8  1000.0               Unknown

 === 178 ===
enc_col = dupl1_v_886
dec_col = dupl1_v_886_B
         enc
0        1.0
1     1000.0
2     1000.0
3        1.0
4     1000.0
...      ...
6452  1000.0
6453     2.0
6454   999.0
6455  1000.0
6456  1000.0

[6241 rows x 1 columns]

      enc                   dec
0     1.0   Normal  

5

# map column names back

In [62]:
colmap_merged

,new,old
0,BL_Age,v_3176
1,BL_Gender,v_341
2,BL_Ethnicity,v_342
3,BL_AdmissionComputedNewCat,BL_AdmissionComputedNewCat
4,BL_EmergencyAdmission,v_3078
...,...,...
223,CR_Compli_IntracerebBleed,v_3562
224,CR_Compli_IschemicStroke,v_3563
225,CR_Compli_CIM,v_3565
226,CR_Compli_CIP,v_3566


In [63]:
data_dec = data_dec.rename(columns = dict(zip(colmap_merged.old, colmap_merged.new)))
data_dec.head()

,BL_Age,BL_Gender,BL_Ethnicity,BL_AdmissionComputedNewCat,BL_EmergencyAdmission,BL_DaysInfectionContactNewCat,BL_StageDiagnosis,BL_UCstartday_bit,BL_UCstartdayNewCategories,BL_COstartday_bit,...,BL_Urine_KetoneBodies,BL_Urine_Bilirubin,BL_Urine_Urubilinogen,CO_Compli_IschemicStroke,CO_Compli_CIM,CR_Compli_IntracerebBleed,CR_Compli_IschemicStroke,CR_Compli_CIM,CR_Compli_CIP,CR_Compli_Seizure
0,36 - 45 years,Female,Unknown,5,No,NaN,Uncomplicated phase,quoted,3,not quoted,...,Not determined/ Unknown,Not determined/ Unknown,Not determined/ Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56 - 65 years,Male,Asian & Pacific Islander,3,No,10,Uncomplicated phase,quoted,3,not quoted,...,Not determined/ Unknown,Not determined/ Unknown,Not determined/ Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36 - 45 years,Female,Unknown,5,No,5,Uncomplicated phase,quoted,5,not quoted,...,Not determined/ Unknown,Not determined/ Unknown,Not determined/ Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,46 - 55 years,Female,Unknown,2,No,5,Uncomplicated phase,quoted,2,not quoted,...,Negativ,Negativ,Negativ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,46 - 55 years,Male,Caucasian,NaN,NaN,3,Uncomplicated phase,quoted,-3,not quoted,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# save cleaned and decoded data

In [66]:
data_dec.to_pickle("../../data/processed/leoss_decoded.pkl", compression="zip")
data_dec.to_csv("../../data/processed/leoss_decoded.tsv", sep="\t")

# create sweetviz plot with all variables

In [141]:
# divide data into baseline and potential endpoints

In [201]:
# potential endpoints
c1 = data_dec.columns.str.contains("startday")
c2 = data_dec.columns.str.contains("Duration")
c3 = data_dec.columns.str.contains("ECMO")
c4 = data_dec.columns.str.contains("LastKnownStatus")
c5 = data_dec.columns.str.contains("^CR_")
c_boolmask = c1 | c2 | c3 | c4 | c5
c_endpoints =  data_dec.columns[c_boolmask]
list(c_endpoints)

['BL_UCstartday_bit',
 'BL_UCstartdayNewCategories',
 'BL_COstartday_bit',
 'BL_COstartdayNewCategories',
 'BL_CRstartday_bit',
 'BL_CRstartdayNewCategories',
 'BL_REstartday_bit',
 'BL_REstartdayNewCategories',
 'BL_Duration_InpatientstayNewCat',
 'BL_Duration_ICUStayNewCat',
 'BL_Duration_IMCUstay',
 'BL_Duration_VentilationNewCat',
 'BL_ECMO',
 'BL_LastKnownStatus',
 'CR_Symp_Delirium',
 'CR_Symp_DeliriumUnknown',
 'CR_Symp_Headache',
 'CR_Symp_HeadacheUnknown',
 'CR_Symp_Otherneuro',
 'CR_Symp_OtherNeuroUnknown',
 'CR_Compli_IntracerebBleed',
 'CR_Compli_IschemicStroke',
 'CR_Compli_CIM',
 'CR_Compli_CIP',
 'CR_Compli_Seizure']

In [202]:
data_dec_endpoints = data_dec.filter(items = c_endpoints, axis=1)
data_dec_endpoints.head()

,BL_UCstartday_bit,BL_UCstartdayNewCategories,BL_COstartday_bit,BL_COstartdayNewCategories,BL_CRstartday_bit,BL_CRstartdayNewCategories,BL_REstartday_bit,BL_REstartdayNewCategories,BL_Duration_InpatientstayNewCat,BL_Duration_ICUStayNewCat,...,CR_Symp_DeliriumUnknown,CR_Symp_Headache,CR_Symp_HeadacheUnknown,CR_Symp_Otherneuro,CR_Symp_OtherNeuroUnknown,CR_Compli_IntracerebBleed,CR_Compli_IschemicStroke,CR_Compli_CIM,CR_Compli_CIP,CR_Compli_Seizure
0,quoted,3,not quoted,NaN,not quoted,NaN,not quoted,NaN,12,0,...,not quoted,not quoted,not quoted,not quoted,not quoted,NaN,NaN,NaN,NaN,NaN
1,quoted,3,not quoted,NaN,not quoted,NaN,quoted,5,12,0,...,not quoted,not quoted,not quoted,not quoted,not quoted,NaN,NaN,NaN,NaN,NaN
2,quoted,5,not quoted,NaN,not quoted,NaN,not quoted,NaN,5,0,...,not quoted,not quoted,not quoted,not quoted,not quoted,NaN,NaN,NaN,NaN,NaN
3,quoted,2,not quoted,NaN,not quoted,NaN,not quoted,NaN,11,0,...,not quoted,not quoted,not quoted,not quoted,not quoted,NaN,NaN,NaN,NaN,NaN
4,quoted,-3,not quoted,NaN,not quoted,NaN,not quoted,NaN,0,0,...,not quoted,not quoted,not quoted,not quoted,not quoted,NaN,NaN,NaN,NaN,NaN


In [203]:
cols_to_int = [
    'BL_UCstartdayNewCategories',
    'BL_COstartdayNewCategories',
    'BL_CRstartdayNewCategories',
    'BL_REstartdayNewCategories',
    'BL_Duration_InpatientstayNewCat',
    'BL_Duration_ICUStayNewCat',
    # 'BL_Duration_IMCUstay',
    'BL_Duration_VentilationNewCat',
    ]
data_dec_endpoints[cols_to_int] = data_dec_endpoints[cols_to_int].apply(pd.to_numeric)

In [204]:
no_endpoints = list(set(data_dec.columns) - set(c_endpoints))
data_dec_noendpoints = data_dec.filter(items = no_endpoints, axis=1)

In [205]:
import sweetviz
my_report = sweetviz.analyze(data_dec_endpoints, pairwise_analysis="on")
my_report.show_html(filepath = "../../reports/leoss_swtvz_v__endpoints.html", layout = "vertical")

Feature: BL_Duration_ICUStayNewCat           |███▉      | [ 39%]   14:14 -> (21:59 left)
Done! Use 'show' commands to display/save.   |██████████| [100%]   00:01 -> (00:00 left)
Report ../../reports/leoss_swtvz_v__endpoints.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [216]:
d1 = data_dec_noendpoints.columns.str.contains("Period")
d2 = data_dec_noendpoints.columns.str.contains("Days")
d3 = data_dec_noendpoints.columns.str.contains("AdmissionComputed")
d = d1 | d2 | d3
d_cnames = data_dec_noendpoints.columns[d]
print(d_cnames)
data_dec_noendpoints[d_cnames] = data_dec_noendpoints[d_cnames].apply(pd.to_numeric)

Index(['BL_AdmissionComputedNewCat', 'BL_ObservationalPeriodNewCat',
       'BL_DaysInfectionContactNewCat'],
      dtype='object')


In [217]:

my_report = sweetviz.analyze(data_dec_noendpoints, pairwise_analysis="on")
my_report.show_html(filepath = "../../reports/leoss_swtvz_v__data_dec_noendpoints.html", layout = "vertical")


Feature: BL_ObservationalPeriodNewCat        |████████▉ | [ 89%]   04:31 -> (00:32 left)
Done! Use 'show' commands to display/save.   |██████████| [100%]   00:48 -> (00:00 left)
Report ../../reports/leoss_swtvz_v__data_dec_noendpoints.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [213]:
data_dec_noendpoints.dtypes

BL_Symp_OtherNeuro                 object
BL_Comorb_ChronicKidneyDisease     object
BL_Comorb_CarotidArtDisease        object
BL_Treat_Statines                  object
BL_Comorb_PriorNeuroDiagnosis      object
                                    ...  
BL_KnownColonization_Unknown       object
BL_Symp_RedEyeUnknown              object
BL_Comorb_DustMiteSensitization    object
BL_Symp_ExcessiveTirednessUnkn     object
BL_DaysInfectionContactNewCat      object
Length: 196, dtype: object

In [214]:
data_dec_noendpoints.infer_objects().dtypes

BL_Symp_OtherNeuro                 object
BL_Comorb_ChronicKidneyDisease     object
BL_Comorb_CarotidArtDisease        object
BL_Treat_Statines                  object
BL_Comorb_PriorNeuroDiagnosis      object
                                    ...  
BL_KnownColonization_Unknown       object
BL_Symp_RedEyeUnknown              object
BL_Comorb_DustMiteSensitization    object
BL_Symp_ExcessiveTirednessUnkn     object
BL_DaysInfectionContactNewCat      object
Length: 196, dtype: object